In [ ]:
import pandas as pd

def get_amazon_data_dict(dataset_path):
    data = pd.read_csv(dataset_path)
    print('========== Start Amazon ==========')
    n_users, n_items, n_cates = data["user_id"].max(), data["item_id"].max(), data["cate_id"].max()

    # features = [SparseFeature("target_item", vocab_size=n_items + 2, embed_dim=8), SparseFeature("target_cate", vocab_size=n_cates + 2, embed_dim=8), SparseFeature("user_id", vocab_size=n_users + 2, embed_dim=8)]
    # target_features = features
    features = [SparseFeature("user_id", vocab_size=n_users + 2, embed_dim=8)]
    target_features = [SparseFeature("target_item", vocab_size=n_items + 2, embed_dim=8),
                       SparseFeature("target_cate", vocab_size=n_cates + 2, embed_dim=8)]
    history_features = [
        SequenceFeature("history_item", vocab_size=n_items + 2, embed_dim=8, pooling="concat", shared_with="target_item"),
        SequenceFeature("history_cate", vocab_size=n_cates + 2, embed_dim=8, pooling="concat", shared_with="target_cate")
    ]

    print('========== Create sequence features ==========')
    train, val, test = create_seq_features(data, seq_feature_col=['item_id', 'cate_id'], drop_short=3)

    print('========== Generate input dict ==========')
    train = df_to_dict(train)
    val = df_to_dict(val)
    test = df_to_dict(test)

    train_y, val_y, test_y = train["label"], val["label"], test["label"]

    del train["label"]
    del val["label"]
    del test["label"]
    train_x, val_x, test_x = train, val, test
    return features, target_features, history_features, (train_x, train_y), (val_x, val_y), (test_x, test_y)